The big picture idea here is that I want to build a machine learning model that can predict when a county will have a high (>75th percentile) rate of COVID fatalities.  I have a wide range of data from countyhealthrankings.org (from 2019), the Social Vulnerability Index (2018) that I will use to predict.

Then I will pull out the most important factors of that model. We have a lot of data that is moderately correlated with COVID fatality rates and often highly correlated with each other. So the machine learning model will help tease out what is really important within the raw data, in a way that minimizes my own biases.

First, a bunch of imports:

In [24]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import roc_auc_score
from COVID_data import prepare_model_data

from sklearn.linear_model import Lasso, LogisticRegressionCV, LassoLarsCV, ElasticNetCV
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import pprint
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

from prettytable import PrettyTable


class config:
    USE_CACHE = True
    CACHE_DIR = "/Users/caseydurfee/msds/data_mining_final_project/cache"

from COVID_data import all_data
data = all_data.get_all_data(config)

Let's look at correlations in year 1 of the pandemic versus year 2 and see if there were changes.  Note that we are looking at all factors, including ones with large rates of missing data, which will artificially inflate those correlation scores. Some of these factors won't be used by the model, because I am throwing out any fields where more than 5% of counties have missing data.

Here are the top 10 factors for each year, filtering out correlations with other death rates.

In [ ]:
death_rate_corr = data.corr()['DEATH_RATE_FIRST_YEAR']

# omicron, alpha, delta, etc. death rates are not interesting here
death_cols = list(data.filter(regex = 'DEATH'))

y1_corr = death_rate_corr.reindex(death_rate_corr.abs().sort_values(ascending=False).index) \
    .drop(death_cols)

print(y1_corr[-10:].to_string())

In [ ]:
death_rate_corr = data.corr()['DEATH_RATE_SECOND_YEAR']

death_cols = list(data.filter(regex = 'DEATH'))

y2_corr = death_rate_corr.reindex(death_rate_corr.abs().sort_values(ascending=False).index) \
    .drop(death_cols)

print(y2_corr[-10:].to_string())

Let's take the top 10 correlations from year 1 and year 2 and see how they've changed.

In [ ]:
cols_in_either = y1_corr[-10:].index.union(y2_corr[-10:].index)

pt = PrettyTable()
pt.field_names = ['Name', 'Year 1 r^2', 'Year 2 r^2']

rows = []
for c in cols_in_either:
    row = [c, round(y1_corr[c], 3), round(y2_corr[c], 3)]
    # pt.add_row(row)
    rows.append(row)

sorted_rows = sorted(rows, key=lambda x: abs(x[1]) + abs(x[2]), reverse=True)

for x in sorted_rows:
    pt.add_row(x)

print(pt)

Some things to note here:
* there are some metrics that are extremely similar (Years of Potential Life Lost Rate and Age-Adjusted Mortality are basically inverses of Life Expectancy). That's to be expected since they're generated by different teams, and we're going to let the model decide which one is the best for predicting high COVID fatality rates.

* vaccination rates don't correlate strongly with year one fatality rate, which makes sense, given time only flows in one direction. They do show up as a major factor in year two, as expected.

* The correlations in year 2 are all much stronger than year 1. This implies year 2 death rates were more predictable based on data we had before the pandemic (and hence the deaths were more preventable).

Let's look at how the top factors correlate with each other. Are they all just saying the same thing? For each factor, we will print out the max correlation with it.

In [ ]:
death_cols = list(data.filter(regex = 'DEATH'))

top_factors = y2_corr[-20:].index

corrs_with_eachother = data.corr().loc[top_factors, top_factors]

pt = PrettyTable()
pt.field_names = ['Column', 'Highest Correlated With', 'r^2']

rows = []
for col in corrs_with_eachother.columns:
    other_cols = corrs_with_eachother[col].drop(col)
    rows.append([col, other_cols.idxmax(), round(max(other_cols), 3)])


sorted_rows = sorted(rows, key=lambda x: (x[1], 1-x[2]))

for row in sorted_rows:
    pt.add_row(row)

print(pt)


So, there's definitely some redundancy. But SVI's Per Capita estimate and Wikipedia's are less correlated than partial COVID vaccination coverage rate and complete coverage rate.

REPUB_PARTISAN is the only one that isn't strongly correlated with anything else. It's also the only one, besides "% Some College", that isn't a measure of health, wealth, or vaccination levels.


I am going to test a wide range of models and we'll go with whatever performs best.  Using ROC AUC as the scoring metric since we have unbalanced classes.

We will drop all metrics that are missing more than 5% of their values. There are about 30 metrics that we don't have good coverage for. If we limit to counties over 50,000 people, we can an additional 10 or so metrics, and possibly lose some noise introduced by imputation done by the sources of the data.

In [ ]:
SEED = 2718
ITERS = 20000

test_models = [ 
    RandomForestClassifier(random_state=SEED),
    SVC(random_state=SEED),
    LogisticRegressionCV(random_state=SEED),
    RidgeClassifierCV(),
    AdaBoostClassifier(random_state=SEED),
    BaggingClassifier(random_state=SEED),
    GradientBoostingClassifier(random_state=SEED),
    LinearSVC(random_state=SEED),
    LassoLarsCV(normalize=False),
    ElasticNetCV(random_state=SEED)
]
roc_auc_scorer = make_scorer(roc_auc_score)

Training on year 1 and year 2 data combined.

In [ ]:
## taken from https://stackoverflow.com/questions/53784971/how-to-disable-convergencewarning-using-sklearn
@ignore_warnings(category=ConvergenceWarning)
def train_combined():
    df, X, y = prepare_model_data.get_train_data(data, year=False)

    roc_scores = {}
    max_roc = 0
    for model in test_models:
        model_name = str(model.__class__)

        mean_score = cross_val_score(model, X, y, scoring=roc_auc_scorer).mean()

        if model_name not in roc_scores:
            roc_scores[model_name] = 0.0
        roc_scores[model_name] += mean_score
        if mean_score > max_roc:
            best_model = model
            max_roc = mean_score
    return roc_scores

roc_scores = train_combined()
combo_roc = pd.Series(roc_scores)
print(">>>>Combined year 1 and year 2")
print(combo_roc.sort_values())

@ignore_warnings(category=ConvergenceWarning)
def fit_yearly_models(iters = 1):
    for year in [1,2]:
        df, X, y = prepare_model_data.make_train_test(data, year=year, min_pop=50000, split=False)

        roc_scores = {}
        max_roc = 0.0
        best_model = None
        for i in range(iters):
            for model in test_models:
                model_name = repr(model.__class__)
                
                mean_score = cross_val_score(model, X, y, scoring=roc_auc_scorer).mean()

                if model_name not in roc_scores:
                    roc_scores[model_name] = 0.0
                roc_scores[model_name] += mean_score
                if mean_score > max_roc:
                    best_model = model
                    max_roc = mean_score

        sz = pd.Series(roc_scores)
        print(f">>>> mean scores for year {year}")
        print(sz / iters)

fit_yearly_models()


The fit on just the second year was much better across the board than the first year, or both years combined. To me, that implies that 2nd year deaths are more predictable than first year desaths.

Looks like ElasticNetCV and LassoLarsCV are our big winners. They both apply regularization, which punishes models for being too complex, leading to them only using a subset of the factors we have. This is perfect for our purposes, since we're trying to pull out the top factors that matter the most.

Because there's not a huge difference between them, I selected LassoLarsCV for the final models because it has less trouble converging on a solution.

now, let's use permutation importance to figure out what really matters to these models that we've built.

In [ ]:
from sklearn.inspection import permutation_importance

df, X, y = prepare_model_data.get_train_data(data, year=1)

y1_model = LassoLarsCV(normalize=False).fit(X, y)

result = permutation_importance(y1_model, X, y, n_repeats=100)

y1_importance = pd.Series(result.importances_mean, index=df.columns)

disp = y1_importance.reindex(y1_importance.abs().sort_values(ascending=False).index)

print("Year one")
print(disp[disp > 0][:10])

The "% Vaccinated (CHR)" measure is actually the standard childhood vaccines, not the COVID vaccine.

In [ ]:
df, X, y = prepare_model_data.get_train_data(data, year=2)

y2_model = LassoLarsCV(normalize=False).fit(X, y)

result = permutation_importance(y2_model, X, y, n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_importance = perm_importances.reindex(perm_importances.abs().sort_values(ascending=False).index)

print("Year two")
print(y2_importance[y2_importance > 0][:10])

These importances don't tell us the direction of the correlation (whether they increase or decrease the likelihood of a county having high COVID rates.) We can unite this data with the correlation data to show all factors that mattered during the pandemic. Let's see which factors showed up in one of the models and had a big change in correlation.

In [ ]:
summary = y1_importance.rename('Y1_IMPORTANCE').to_frame()\
    .join(y1_corr.rename("Y1_CORR"))\
    .join(y2_importance.rename('Y2_IMPORTANCE'))\
    .join(y2_corr.rename("Y2_CORR"))

summary['CORR_DIFF'] = summary['Y2_CORR'] - summary['Y1_CORR']

#summary[(summary.Y1_IMPORTANCE > 0) & (summary.Y2_IMPORTANCE > 0)].sort_values(by='Y2_IMPORTANCE', ascending=False)

In [ ]:
summary[(summary.Y1_IMPORTANCE > 0) | (summary.Y2_IMPORTANCE > 0)].sort_values(by='CORR_DIFF', key=lambda x: abs(x), ascending=False)[:10]

Let's see if ElasticNetCV would give us different factors for year two.

In [ ]:
df, X, y = prepare_model_data.get_train_data(data, year=2)

enet_model = ElasticNetCV().fit(X, y)

result = permutation_importance(enet_model, X, y, n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_enet = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [ ]:
enet_vs_lars = y2_enet.rename('Y2_ENET').to_frame()\
    .join(y2_importance.rename('Y2_LARS'))

enet_vs_lars['ENET_RANK'] = enet_vs_lars['Y2_ENET'].rank(ascending=False)
enet_vs_lars['LARS_RANK'] = enet_vs_lars['Y2_LARS'].rank(ascending=False)

enet_vs_lars["RANK_CHANGE"] = abs(enet_vs_lars['ENET_RANK'] - enet_vs_lars['LARS_RANK'])


enet_vs_lars[((enet_vs_lars.Y2_ENET > 0) | (enet_vs_lars.Y2_LARS > 0)) & (enet_vs_lars.ENET_RANK < 21)].sort_values(by='ENET_RANK')

It looks like the rankings are remarkably consistent between the two models. The order is slightly different, but the top 20 factors are the same.

Let's try a couple of the less accurate models and see what factors they deem important. 

First off, let's try LogisticRegressionCV. Because it doesn't have a regularization penalty, it will use all 80 factors in the model, so we will only display the top 20.

In [ ]:
lrcv_model = LogisticRegressionCV(max_iter=20000).fit(X, y)

result = permutation_importance(lrcv_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_lrcv = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [ ]:
print(y2_lrcv[-20:])

We saw that LARS and ENET had the same top 20 factors. What about LRCV? How much overlap is there?

In [ ]:
enet_vs_lars[-20:].index.intersection(y2_lrcv[-20:].index)


In [ ]:
enet_vs_lars[-10:].index.intersection(y2_lrcv[-10:].index)

Let's compare with LinearSVC.

In [ ]:
svc_model = LinearSVC(max_iter=50000).fit(X,y)

result = permutation_importance(svc_model, X, y, 
            n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_svc = perm_importances.reindex(perm_importances.abs().sort_values().index)

In [ ]:
enet_vs_lars[-20:].index.intersection(y2_svc[-20:].index)

In [ ]:
enet_vs_lars[-10:].index.intersection(y2_svc[-10:].index)

All four models identified `2016 Repub Vote Share` as the most important factor and `Physically Unhealthy Days (CHR)` as also being important.

We've been looking at all counties. However, many of the metrics we have are imputed by the sources we got them from (SVI, County Health Rankings, CDC, etc.) Those data sources provide confidence intervals on their estimations but we're not using those.

So it makes sense to look at just big US counties (over 50,000 population)

In [ ]:
df, X, y = prepare_model_data.make_train_test(data, year=2, min_pop=50000, split=False)

y2_bigcounty_model = LassoLarsCV(normalize=False).fit(X, y)

result = permutation_importance(y2_bigcounty_model, X, y, n_repeats=100)

perm_importances = pd.Series(result.importances_mean, index=df.columns)

y2_bigcounty_importance = perm_importances.reindex(perm_importances.abs().sort_values(ascending=False).index)

print("Year two - Big Counties Only")
print(y2_bigcounty_importance[y2_bigcounty_importance > 0][:20])

Once again REPUB_PARTISAN is the strongest factor used by the model to predict high COVID fatality rates.